In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
from data_api import query_date
from decimal import *
from dateutil import tz

pd.set_option('display.max_colwidth', -1)

start_time = Decimal(datetime.datetime(2020, 5, 26, 0, 0, 0).timestamp())
end_time = Decimal(datetime.datetime.utcnow().timestamp())
response = query_date('TSLA', start_time, end_time)
data = pd.DataFrame.from_dict(response)

<ipython-input-91-b23c6206ec66>:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [92]:
to_zone = tz.gettz('America/New_York')
time_offset = 14400 # 4 hours in seconds
data['timestamp'] = data['timestamp'].apply(lambda x: (float(str(x))-time_offset)*1000)
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data = data.set_index('timestamp')
data.index = pd.to_datetime(data_timeidx.index, unit='s')

In [100]:
def isolate_price_changes(data):
    data = data.reset_index()
    last_row = None
    last_index = None
    to_drop = []
    for index, row in data.iterrows():
        if index == 0:
            last_row = row
            last_index = index
            continue
        if last_row['last'] == row['last'] and last_row['last_size'] == row['last_size']:
            to_drop.append(index)
        last_row = row
        last_index = index
    data.drop(to_drop)
    data = data.set_index('timestamp')
    return data
data = isolate_price_changes(data)

finished parsing


In [117]:
def compute_normal_bar(df):
    if len(df) == 0:
        return

    volume = 0
    high = -1.0
    low = 1000000.0
    price_sum = 0
    vwap = 0
    open = df.head(1)['last']
    close = df.tail(1)['last']
    
    for index, tick in df.iterrows():
        price_sum += float(tick['last']) * float(tick['last_size'])
        if float(tick['last']) < low:
            low = float(tick['last'])
        if float(tick['last']) > high:
            high = float(tick['last'])
        volume += int(tick['last_size']) * 100
    if volume != 0:
        vwap = float(price_sum) / float(volume)
        bars = "h: %.2f" % high + " l: %.2f" % low + " o: %.2f" % float(open) + " c: %.2f" % float(close) + " vwap: %.2f" % vwap + " v: " + str(volume)
    else:
        bars = "h: %.2f" % high + " l: %.2f" % low + " o: %.2f" % float(open) + " c: %.2f" % float(close) + " v: " + str(volume)        
    return bars

In [108]:
#time bars
data_time_groups = data.groupby(pd.Grouper(freq='60Min'))
data_time_bars = data_time_groups.apply(compute_normal_bar)
print(data_time_bars)

timestamp
2020-06-25 03:00:00    h: 951.00 l: 951.00 o: 951.00 c: 951.00 vwap: 9.51 v: 800    
2020-06-25 04:00:00    h: 968.79 l: 946.85 o: 950.00 c: 968.79 vwap: 9.58 v: 34700  
2020-06-25 05:00:00    h: 968.79 l: 963.88 o: 968.79 c: 963.88 vwap: 9.67 v: 25200  
2020-06-25 06:00:00    h: 968.44 l: 962.35 o: 963.88 c: 962.46 vwap: 9.65 v: 36000  
2020-06-25 07:00:00    h: 967.00 l: 952.15 o: 965.00 c: 953.30 vwap: 9.60 v: 155000 
2020-06-25 08:00:00    h: 964.00 l: 949.90 o: 962.98 c: 952.00 vwap: 9.53 v: 349500 
2020-06-25 09:00:00    h: 959.90 l: 937.15 o: 952.00 c: 948.45 vwap: 9.47 v: 2700000
2020-06-25 10:00:00    h: 967.79 l: 940.95 o: 948.32 c: 960.12 vwap: 9.56 v: 4400300
2020-06-25 11:00:00    h: 966.60 l: 955.01 o: 960.12 c: 965.06 vwap: 9.61 v: 2792600
2020-06-25 12:00:00    h: 975.47 l: 965.06 o: 965.06 c: 973.49 vwap: 9.72 v: 3140200
2020-06-25 13:00:00    h: 975.20 l: 968.23 o: 973.49 c: 974.07 vwap: 9.73 v: 2366000
2020-06-25 14:00:00    h: 975.65 l: 968.93 o: 974.07 c:

In [121]:
#tick bars
#TODO: what is a tick exactly, a price change?
def tick_group(index):
    return (int(index / 1000))
# data = data.reset_index()
data_tick_groups = data.groupby(tick_group)
data_tick_bars = data_tick_groups.apply(compute_normal_bar)
print(data_tick_bars)
data = data.set_index('timestamp')

0      h: 968.79 l: 946.85 o: 951.00 c: 962.58 vwap: 9.63 v: 107800
1      h: 967.00 l: 952.15 o: 962.58 c: 953.09 vwap: 9.62 v: 100700
2      h: 964.00 l: 950.11 o: 953.09 c: 953.50 vwap: 9.53 v: 146700
3      h: 955.83 l: 950.10 o: 953.50 c: 953.01 vwap: 9.53 v: 156300
4      h: 954.95 l: 949.90 o: 953.01 c: 952.11 vwap: 9.52 v: 144800
                                   ...                             
222    h: 962.94 l: 958.26 o: 962.94 c: 961.20 vwap: 9.60 v: 128100
223    h: 961.71 l: 960.21 o: 961.20 c: 960.77 vwap: 9.61 v: 143400
224    h: 961.90 l: 959.74 o: 960.77 c: 960.00 vwap: 9.61 v: 142000
225    h: 960.84 l: 958.01 o: 960.00 c: 958.01 vwap: 9.60 v: 142700
226    h: 959.10 l: 955.00 o: 958.01 c: 955.90 vwap: 9.57 v: 67800 
Length: 227, dtype: object
